# QUESTION 5

## 

## Install the required packages

In [1]:
import pandas as pd
from statistics import mean
import scipy.stats as stats
import numpy as np
import scipy
import seaborn as sns
from random import sample
from scipy.stats import f_oneway
import warnings
warnings.filterwarnings("ignore")
import pingouin as pg
from scipy.stats import mannwhitneyu

## 

## Loading the dataset

In [2]:
df = pd.read_csv("titanic_dataset.csv")

In [3]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
df.shape

(891, 12)

In [5]:
df.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## 

## A. Apply hypothesis test for mean on any column of the dataset to check null hypothesis is accepted or not

In [7]:
fare_mean = mean(df['Fare'])
print(fare_mean)

32.204207968574636


The mean average of Fare is 32

In [8]:
#performing one sample t-test
stats.ttest_1samp(a=df['Fare'], popmean=32)

Ttest_1sampResult(statistic=0.12266271558913089, pvalue=0.9024018818220448)

A we can see from the result above, we cannot reject the test's null hypothesis because the p-value is (0.9024), which is higher than alpha (0.05)

## 

## Apply the hypothesis test for variance to check if two samples of data from the 'Fare' column have equal variance

In [9]:
x = df['Fare'].sample(150).to_numpy()
y = df['Fare'].sample(200).to_numpy()

In [10]:
#calculate the F-test statistic 

f_test = np.var(x, ddof=1)/np.var(y, ddof=890)

In [11]:
#define the degrees of freedom numerator 
dfn = 1

#define the degrees of freedom denominator 
dfd = 890 

In [12]:
#find the p-value of the F-test statistic 
p_value = 1-scipy.stats.f.cdf(f_test, dfn, dfd)

print(f_test, p_value)

0.0 1.0


## 

## B. Apply one-way and two-way ANOVA on any columns of the DatFrame to check whether the null hypothesis is accepted or not.

### One-way ANOVA Null Hypothesis

In [21]:
fare_sample1 = df['Fare'].sample(40)
fare_sample2 = df['Fare'].sample(40)
fare_sample3 = df['Fare'].sample(40)



In [22]:
f_oneway(fare_sample1, fare_sample2, fare_sample3)

F_onewayResult(statistic=1.8563565621652722, pvalue=0.16081322369006637)

## 

## Two-way ANOVA Null Hypothesis

#### Example 1

In [17]:
sample_two_anova = df.loc[:,["Survived", "Pclass", "Embarked"]]
sample_two_anova = sample_two_anova.sample(150)


In [18]:
test_aov = pg.anova(sample_two_anova, dv="Survived", between=["Pclass", "Embarked"], detailed=True)
test_aov

,Source,SS,DF,MS,F,p-unc,np2
0,Pclass,3.228579,2.0,1.614289,8.811046,0.000248,0.111095
1,Embarked,3.775068,2.0,1.887534,10.302458,0.000067,0.127502
2,Pclass * Embarked,1.311170,4.0,0.327792,1.789143,0.134311,0.048304
3,Residual,25.832893,141.0,0.183212,NaN,NaN,NaN


In [19]:
#check the critical F-value to decide whether the null hypothesis is accepted or not.
scipy.stats.f.ppf(q=1-0.05, dfn=1, dfd=890)

3.8519283899238723

## 

#### Example 2

In [118]:
sample_two_anova2 = df.loc[:,["Fare", "Age", "Embarked"]]
sample_two_anova2 = sample_two_anova2.sample(15)


In [119]:
test_aov2 = pg.anova(sample_two_anova2, dv="Fare", between=["Age", "Embarked"], detailed=True)
test_aov2

,Source,SS,DF,MS,F,p-unc,np2
0,Age,3346.910962,8.0,418.363870,4.951052,0.334944,0.975375
1,Embarked,3346.823268,1.0,3346.823268,39.607376,0.100317,0.975374
2,Age * Embarked,2104.534040,8.0,263.066755,3.113216,0.413574,0.961398
3,Residual,84.500000,1.0,84.500000,NaN,NaN,NaN


In [120]:
#check the critical F-value to decide whether the null hypothesis is accepted or not.
scipy.stats.f.ppf(q=1-0.05, dfn=495, dfd=395)

1.1710681887942844

## 

## 

## C. Make a critical analysis and present a report on how and why the null and alternative hypotheses are accepted or rejected based on the results obtained from the above two tasks (a and b).

### Hypothesis test for mean analysis

The one-sample test usually is used by stasticians to calculate the population's mean, which is equal to a given value. Also, the sample- test used to check if the null hypothesis is accepted or rejected based on the p-value's result.
For example, in our analysis the mean average of Fare is 32 and the p-value is (0.9024), therefore, the test's null hypothesis cannot be rejected because the p-value is greater than alpha (0.05) and the deviation from it is not statically significant.



## 

### One-way ANOVA Null Hypothesis analysis

One-way ANOVA is another statistic calculation test that is used to verify whether there is any correlation between the mean values of more than one group.
Let's say that in our analyses done above with the 3 samples regarding fare column, we can see that the results show us that the p-value is equal to (0.160) and while the F-statistic is equal to (1.856). Therefore, due to the p-value is greather than (0.05), we would accept the null hypothesis in this case.








## 

### Two-way ANOVA Null Hypothesis analysis¶

#### EXAMPLE 1

In contrast to one-way ANOVA, the two-way ANOVA test is used to verify how to two factors affect a variable and if there are any relationship between the two factors and the response variable.
Let's say that in the titanic's dataset, the two-way ANOVA calculations show us that the critical F-value generated is (3.85), and the two ANOVA statical F-value combining "Pclass" and "Embarked" from the stastical dataframe above is (1.78), which is less than the critical F-value. Therefore, the null hypothesis is rejected because there is not relationship between "Pclass" and "Embarked" to survived passengers.







#### EXAMPLE 2

In the second example, the F-value generated is (1.17), and the two ANOVA statical F-value combined with Age and Embarked is (3.11), which is greater than the critical F-value. Therefore, the null hypothesis will not be rejected because both variables have a strong relationship with the "Fare" variable. 
